# MODEL B: 299x299 Model

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os

In [3]:
datagen = ImageDataGenerator(rescale = .1/255, rotation_range = 40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_generator = datagen.flow_from_directory(
    'train',
    target_size=(299, 299),
    class_mode='categorical',
    batch_size=400  # Set batch size to match the total number of images
)

Found 9112 images belonging to 5 classes.


In [4]:
model = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), weights='imagenet', include_top=False)

# Freeze the weights of all the layers except for the last one
for layer in model.layers[:-1]:
    layer.trainable = False

# Add a new output layer
x = tf.keras.layers.Flatten()(model.output)
x = tf.keras.layers.Dense(128,"relu")(x)
x = tf.keras.layers.Dense(5, "softmax")(x)

# Compile the model
model = tf.keras.Model(model.input, x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2023-10-19 16:01:56.788994: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-10-19 16:01:56.789030: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-10-19 16:01:56.789043: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-10-19 16:01:56.789119: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-19 16:01:56.789289: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
checkpoint_path = "checkpoints/modelB_299.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [8]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [6]:
model.load_weights(checkpoint_path)

In [7]:
# Train the model
# TODO: add checkpointing
# model.fit(train_generator, epochs=30, callbacks=[cp_callback])

In [8]:
model.save("models/modelB_299.h5")

/opt/homebrew/Caskroom/miniconda/base/envs/GG_2907/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
